In [7]:
import os
import glob
import pandas as pd
import numpy as np

from pathlib import Path

OUT = Path("../outputs")
OUT.mkdir(parents=True, exist_ok=True)

print("Working directory:", Path.cwd())
print("Outputs directory:", OUT.resolve())

Working directory: c:\Users\Yash Verma\Desktop\College\5th Sem\Data Science\Project\PG\notebooks
Outputs directory: C:\Users\Yash Verma\Desktop\College\5th Sem\Data Science\Project\PG\outputs


In [8]:
# Configuration: change if your CSV folder is different
DATA_DIR = Path("../data/")   # folder containing the 8 CSVs
CLEANED_PATH = OUT / "cleaned_data.csv"
force_merge = False   # set True to re-merge raw CSVs even if cleaned_data.csv exists

print("Data dir:", DATA_DIR.resolve() if DATA_DIR.exists() else "NOT FOUND")
print("Cleaned file path:", CLEANED_PATH)


Data dir: C:\Users\Yash Verma\Desktop\College\5th Sem\Data Science\Project\PG\data
Cleaned file path: ..\outputs\cleaned_data.csv


In [9]:
# Load or merge CSVs
if CLEANED_PATH.exists() and not force_merge:
    print("Loading existing cleaned file:", CLEANED_PATH)
    df = pd.read_csv(CLEANED_PATH, low_memory=False)
else:
    # find CSVs
    csv_files = sorted(list(DATA_DIR.glob("*.csv")))
    if len(csv_files) == 0:
        raise FileNotFoundError(f"No CSVs found in {DATA_DIR}. Place your CIC-IDS-2017 CSVs there.")
    print(f"Found {len(csv_files)} CSV files. Merging...")

    parts = []
    for i, p in enumerate(csv_files, 1):
        print(f"  [{i}/{len(csv_files)}] Reading {p.name}")
        parts.append(pd.read_csv(p, low_memory=False))
    df = pd.concat(parts, axis=0, ignore_index=True, sort=False)
    print("Merged shape:", df.shape)

    # write initial merged file (optional)
    try:
        df.to_csv(CLEANED_PATH, index=False)
        print("Merged file saved to:", CLEANED_PATH)
    except Exception as e:
        print("Warning: could not save merged file:", e)


Found 8 CSV files. Merging...
  [1/8] Reading Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
  [2/8] Reading Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
  [3/8] Reading Friday-WorkingHours-Morning.pcap_ISCX.csv
  [4/8] Reading Monday-WorkingHours.pcap_ISCX.csv
  [5/8] Reading Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
  [6/8] Reading Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
  [7/8] Reading Tuesday-WorkingHours.pcap_ISCX.csv
  [8/8] Reading Wednesday-workingHours.pcap_ISCX.csv
Merged shape: (2830743, 79)
Merged file saved to: ..\outputs\cleaned_data.csv


In [10]:
# Normalize column names (strip and collapse multiple spaces)
df.columns = [(" ".join(str(c).split())).strip() for c in df.columns]

print("Number of columns:", len(df.columns))
print("Sample columns:", df.columns.tolist()[:25])
# Save column list for inspection
pd.Series(df.columns.tolist(), name="columns").to_csv(OUT / "columns_list.csv", index=False)
print("Saved columns_list.csv to outputs/")


Number of columns: 79
Sample columns: ['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min']
Saved columns_list.csv to outputs/


In [11]:
# Missing value summary
missing_frac = df.isna().mean().sort_values(ascending=False)
top_missing = missing_frac.head(20)
print("Top columns by missing fraction (showing 20):")
print(top_missing)

# Drop columns with >30% missing (tunable)
drop_threshold = 0.30
cols_to_drop = missing_frac[missing_frac > drop_threshold].index.tolist()
if cols_to_drop:
    print(f"Dropping {len(cols_to_drop)} columns with >{int(drop_threshold*100)}% missing.")
    df.drop(columns=cols_to_drop, inplace=True)

print("Shape after dropping high-missing columns:", df.shape)


Top columns by missing fraction (showing 20):
Flow Bytes/s                   0.00048
Flow Duration                  0.00000
Destination Port               0.00000
Total Backward Packets         0.00000
Total Length of Fwd Packets    0.00000
Total Length of Bwd Packets    0.00000
Total Fwd Packets              0.00000
Fwd Packet Length Max          0.00000
Fwd Packet Length Min          0.00000
Fwd Packet Length Std          0.00000
Fwd Packet Length Mean         0.00000
Bwd Packet Length Max          0.00000
Bwd Packet Length Min          0.00000
Bwd Packet Length Mean         0.00000
Bwd Packet Length Std          0.00000
Flow Packets/s                 0.00000
Flow IAT Mean                  0.00000
Flow IAT Std                   0.00000
Flow IAT Max                   0.00000
Flow IAT Min                   0.00000
dtype: float64
Shape after dropping high-missing columns: (2830743, 79)


In [12]:
# Replace infinite values with NaN across numeric dtypes
numcols_before = df.select_dtypes(include=[np.number]).columns.tolist()
df[numcols_before] = df[numcols_before].replace([np.inf, -np.inf], np.nan)

# Explicit numeric columns to coerce (edit if your dataset names differ)
numeric_cols = [
    'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
    'Flow Bytes/s', 'Flow Packets/s', 'Average Packet Size', 'Fwd Packet Length Mean',
    'Bwd Packet Length Mean'
]
# Keep only those that actually exist in df
numeric_cols_present = [c for c in numeric_cols if c in df.columns]

print("Coercing to numeric (columns found):", numeric_cols_present)
for c in numeric_cols_present:
    df[c] = pd.to_numeric(df[c], errors='coerce')


Coercing to numeric (columns found): ['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Flow Bytes/s', 'Flow Packets/s', 'Average Packet Size', 'Fwd Packet Length Mean', 'Bwd Packet Length Mean']


In [13]:
# Example domain fixes: negative durations -> NaN
if 'Flow Duration' in df.columns:
    neg_dur = (df['Flow Duration'] < 0).sum()
    if neg_dur:
        print(f"Setting {neg_dur} negative Flow Duration values to NaN")
        df.loc[df['Flow Duration'] < 0, 'Flow Duration'] = np.nan

# Impute numeric NaNs with median (simple and robust)
numeric_cols_all = df.select_dtypes(include=[np.number]).columns.tolist()
print("Numeric columns count:", len(numeric_cols_all))
df[numeric_cols_all] = df[numeric_cols_all].fillna(df[numeric_cols_all].median())
print("Imputed numeric NaNs with median.")


Setting 115 negative Flow Duration values to NaN
Numeric columns count: 78
Imputed numeric NaNs with median.


In [14]:
# Remove exact duplicate rows
dup_count = df.duplicated().sum()
if dup_count:
    print(f"Dropping {dup_count} duplicate rows")
    df = df.drop_duplicates().reset_index(drop=True)
else:
    df = df.reset_index(drop=True)
print("Shape after dedup:", df.shape)


Dropping 308381 duplicate rows
Shape after dedup: (2522362, 79)


In [15]:
# Detect label column (expected 'Label'); try common names otherwise
possible_label_cols = [c for c in df.columns if c.lower().strip() in ('label','class','attack','attack_label')]
label_col = possible_label_cols[0] if possible_label_cols else None

if not label_col:
    # fallback: heuristic: string column with 'BENIGN' samples
    for c in df.columns:
        if df[c].dtype == object:
            s = df[c].dropna().astype(str).str.upper().head(200).tolist()
            if any('BENIGN' in v for v in s):
                label_col = c
                break

if not label_col:
    raise RuntimeError("Label column not found automatically. Edit notebook and set label_col manually.")

print("Using label column:", label_col)
df[label_col] = df[label_col].astype(str).str.strip().str.upper()
df['Attack_Binary'] = df[label_col].apply(lambda x: 'BENIGN' if 'BENIGN' in x else 'ATTACK')

# Save class balance
df['Attack_Binary'].value_counts().to_csv(OUT / "class_balance.csv", header=['count'])
print("Saved class_balance.csv (BENIGN vs ATTACK counts)")


Using label column: Label
Saved class_balance.csv (BENIGN vs ATTACK counts)


In [16]:
# Final save
cleaned_path = OUT / "cleaned_data.csv"
df.to_csv(cleaned_path, index=False)
print("Saved cleaned dataset to:", cleaned_path)

# Basic descriptive summary
desc = df.describe(include='all').T
desc.to_csv(OUT / "descriptive_summary.csv")
print("Saved descriptive_summary.csv")

# Sample head for quick inspection
df.head(5).to_csv(OUT / "sample_head.csv", index=False)
print("Saved sample_head.csv")


Saved cleaned dataset to: ..\outputs\cleaned_data.csv
Saved descriptive_summary.csv
Saved sample_head.csv


In [ ]:
# Quick sanity checks to print in notebook
print("Rows,Cols:", df.shape)
print("Top 10 columns and types:")
print(df.dtypes.head(10))
print("Any NaNs remaining in numeric columns (counts):")
print(df.select_dtypes(include=[np.number]).isna().sum().sort_values(ascending=False).head(15))
print("First 10 unique labels:", df[label_col].unique()[:10])


Rows,Cols: (2522362, 80)
Top 10 columns and types:
Destination Port                 int64
Flow Duration                  float64
Total Fwd Packets                int64
Total Backward Packets           int64
Total Length of Fwd Packets      int64
Total Length of Bwd Packets      int64
Fwd Packet Length Max            int64
Fwd Packet Length Min            int64
Fwd Packet Length Mean         float64
Fwd Packet Length Std          float64
dtype: object
Any NaNs remaining in numeric columns (counts):
Destination Port               0
Flow Duration                  0
Total Fwd Packets              0
Total Backward Packets         0
Total Length of Fwd Packets    0
Total Length of Bwd Packets    0
Fwd Packet Length Max          0
Fwd Packet Length Min          0
Fwd Packet Length Mean         0
Fwd Packet Length Std          0
Bwd Packet Length Max          0
Bwd Packet Length Min          0
Bwd Packet Length Mean         0
Bwd Packet Length Std          0
Flow Bytes/s                   0
dt